DC, for the Employment data analysis Please do the Data preparatin as following:
 
Get the following fields from the CICdata
 
customer_id, TIN, run_date, occupationdesc, occupationstatusdesc, TradeName, employment_duration_in_days (DateHiredTo or run_date - DateHiredFrom), PSICdesc, AnnualMonthlyIndicator, Currency, GrossIncome, max_sanctioned_amt_24m, max_creditcard_limit_24m, max_financed_amt_24m, tot_granted_contracts_cnt_24m, tot_granted_contracts_amt_12m, max_sanctioned_amt_12m, Max_CreditCard_limit_12m, max_financed_amt_12m, tot_granted_contracts_cnt_12m, tot_granted_contracts_amt_12m
 
Get the self declared employment details, application_date,  and MaxSalary information from our internal Salary scaling table using the CustomerID or TIN and application_date = run_date
 
Get the gender, age, region, province of the customer from demographic table using customer_id or TIN and application_date = run_date
 
Get the following fields from the Credolab tables using the customer_id or mobile number and application_date = run date:
Device Price
Device Manufacturer (Apple/ Samsung/ Oppo etc.)
Device Model
Device Age (usually derived by the application_date - earliest apps installation date)
number of job search apps installed (linkedin, jobstreet, glint etc.)
number of professional apps installed (office, gdrive etc)
number of ecommerce apps installed (lazada, shopee, shein, zalora, temu etc.)
number of ride hailing apps installed (grab, moveit etc.)
number of finance apps installed (Banking apps, FinTech apps, eWallet apps)
number of Risky apps (gambling apps, crypto apps, payday loan apps)
number of OTT apps (Netflix, PrimeVideo etc.)
Sowndarya: may be you can share your earlier apps categorisation work with DC for this. To keep things simple, just give him your final app categorisation mapping.

# <div align="center" style="color: #C64242;">**Employement Data with CredoLab Data For Income Estimation**</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import timeit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
import re
from fuzzywuzzy import process
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

c:\Users\DwaipayanChakroborti\Myenv\reporting\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## <div style="color: #C64242; font-family: Monotype Corsiva;"><b> CredoLabData</b> </div>

In [2]:
%%time

# CredoLab Data 

sq = """Select  DISTINCT t1.customerId,t3.crodolabDeviceId, ca.package_name
FROM
`prj-prod-dataplatform.risk_credit_mis.loan_master_table` t1
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` t2
ON t1.digitalLoanAccountId = t2.digitalLoanAccountId
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_credolab_track` t3
ON t2.credolabRefNumber = t3.refno
LEFT JOIN
`prj-prod-dataplatform.credolab_raw.android_credolab_datasets_struct_columns` t4
ON t3.refno = t4.deviceId
inner join
`prj-prod-dataplatform.core_raw.loan_accounts` loan
on loan.CUSTOMERID = t1.customerId
 INNER JOIN
(select deviceId, af.package_name as package_name, af.first_install_time as first_install_time from `prj-prod-dataplatform.credolab_raw.android_credolab_Application`  ,
unnest(Application) as af) ca
ON ca.deviceId = t3.refno
where date_trunc(t1.startApplyDateTime, day) >'2024-04-30'
;"""

df_risk_table_2  = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The rows and columns in the credolab dataframe are:\t {df_risk_table_2.shape}")

Job ID 7e2156e6-7a63-4f98-b833-6c968c9cd606 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in the credolab dataframe are:	 (11853249, 3)
CPU times: total: 2min 18s
Wall time: 22min 31s


In [3]:
sq = """
WITH base_data AS (
  SELECT 
    t1.customerId,
    t3.crodolabDeviceId, 
    ca.package_name
  FROM
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` t1
  LEFT JOIN
    `prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` t2
    ON t1.digitalLoanAccountId = t2.digitalLoanAccountId
  LEFT JOIN
    `prj-prod-dataplatform.dl_loans_db_raw.tdbk_credolab_track` t3
    ON t2.credolabRefNumber = t3.refno
  LEFT JOIN
    `prj-prod-dataplatform.credolab_raw.android_credolab_datasets_struct_columns` t4
    ON t3.refno = t4.deviceId
  INNER JOIN
    `prj-prod-dataplatform.core_raw.loan_accounts` loan
    ON loan.CUSTOMERID = t1.customerId
  INNER JOIN
    (SELECT 
      deviceId, 
      af.package_name AS package_name, 
      af.first_install_time AS first_install_time 
    FROM 
      `prj-prod-dataplatform.credolab_raw.android_credolab_Application`,
      UNNEST(Application) AS af
    ) ca
    ON ca.deviceId = t3.refno
  WHERE 
    DATE_TRUNC(t1.startApplyDateTime, DAY) > '2024-03-31'
),
b AS (
  SELECT DISTINCT customerId, crodolabDeviceId, package_name
  FROM base_data
),
package_parts AS (
  SELECT
    customerId,
    package_name,
    SPLIT(package_name, '.')[SAFE_OFFSET(0)] AS part1,
    SPLIT(package_name, '.')[SAFE_OFFSET(1)] AS part2,
    SPLIT(package_name, '.')[SAFE_OFFSET(2)] AS part3,
    SPLIT(package_name, '.')[SAFE_OFFSET(3)] AS part4,
    SPLIT(package_name, '.')[SAFE_OFFSET(4)] AS part5,
    ARRAY_REVERSE(SPLIT(package_name, '.'))[SAFE_OFFSET(0)] AS last_part
  FROM b
)
SELECT
  package_name,
  part1,
  part2,
  part3,
  part4,
  part5,
  last_part,
  COUNT(customerId) AS cntuniquecustomer
FROM
  package_parts
GROUP BY
  1, 2, 3, 4, 5, 6, 7
ORDER BY
  cntuniquecustomer DESC;
"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')


Job ID 75dfd543-7a38-4231-904d-f9dd159d12b3 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [4]:
data = df.copy()

In [5]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\IncomeEstimation\Data\PackageSplit.xlsx", index = False)

In [6]:
data =    {"category" :['social_media',"social_media","social_media","social_media",'social_media',"social_media",
                        "social_media",'social_media',"social_media","social_media","social_media",'social_media',
                        "social_media","social_media","social_media","social_media",'social_media',"social_media",
                        "social_media","social_media",'social_media',"social_media","social_media",'social_media',
                        "social_media",'social_media',"social_media","social_media","social_media",'social_media',
                        "social_media","social_media",'social_media',"social_media",'social_media',"social_media",
                        "social_media",'social_media',
                        "Health & lifestyle",  "Health & lifestyle",  "Health & lifestyle", "Health & lifestyle","Health & lifestyle",
                        "Health & lifestyle",  "Health & lifestyle","Health & lifestyle",  "Health & lifestyle",  "Health & lifestyle", 
                       "Health & lifestyle",  "Health & lifestyle",  "Health & lifestyle",  "Health & lifestyle",  "Health & lifestyle",
                       "Health & lifestyle",  "Health & lifestyle",  "Health & lifestyle",  "Health & lifestyle",
                        
                        "ecommerce","ecommerce","ecommerce","ecommerce",'ecommerce',"ecommerce","ecommerce","ecommerce","ecommerce",'ecommerce',
                        "ecommerce","ecommerce","ecommerce","ecommerce",'ecommerce', "ecommerce","ecommerce","ecommerce","ecommerce",'ecommerce',
                        
                        'productivity','productivity','productivity','productivity', 'productivity',
                        'productivity','productivity','productivity','productivity','productivity',
                        'productivity','productivity','productivity','productivity','productivity',
                        'productivity','productivity','productivity','productivity', 'productivity',
                        'productivity','productivity','productivity','productivity','productivity',
                        
                        "travel","travel","travel","travel","travel","travel","travel","travel","travel","travel",
                        "travel","travel","travel","travel","travel","travel","travel","travel","travel",
                        "VPN","VPN", "VPN","VPN","VPN","VPN","VPN","VPN","VPN",
                        
                         "finance","finance","finance","finance","finance","finance","finance","finance","finance","finance",
                        "finance","finance","finance","finance","finance","finance","finance","finance","finance","finance",
                       "finance","finance","finance","finance","finance","finance","finance","finance","finance","finance",
                        
                        "games",'games','games','games','games','games','games','games','games','games',"games",'games',
                        'games','games','games','games','games','games','games','games',"games",'games','games','games',
                        'games','games','games','games','games','games','games','games','games','games',
                        
                        "News","News", "News","News","News","News","News","News","News","News",
                        "News","News", "News","News","News","News","News","News","News","News","News",
                        
                        "Podcast", "Podcast", "Podcast", "Podcast", "Podcast", "Podcast", "Podcast", "Podcast","Podcast", "Podcast",
                         "Podcast", "Podcast", "Podcast", "Podcast", "Podcast", "Podcast", "Podcast", "Podcast","Podcast", "Podcast",
                        "Entertainment","Entertainment","Entertainment","Entertainment","Entertainment","Entertainment","Entertainment","Entertainment",
                        "Entertainment","Entertainment","Entertainment","Entertainment","Entertainment","Entertainment","Entertainment","Entertainment"

                     
                       ],
                       
          "apps" :["messanger","facebook","viber","instagram","lite","facebooklite","Telegram","Discord","Pinterest",
                   "bilibili","reddit","Twitter","linkedin", 'Instagram', 'Twitter', 'LinkedIn', 'Snapchat',
                   'TikTok', 'Pinterest', 'WhatsApp', 'Reddit', 'YouTube', 'Tumblr', 'Skype', 
                   'WeChat', 'LINE', 'Viber', 'Signal', 'Twitch', 'VKontakte', 'Weibo', 'Clubhouse', 
                   'Flickr', 'Quora', 'Badoo', 'Meetup', 'KakaoTalk', 'VK',
                   
                   'MyFitnessPal', 'Calm', 'Headspace', 'Fitbit', 'NikeTrainingClub', 'LoseIt', 'Strava', 'Yoga Studio', 
                   'DailyYoga', 'MapMyRun', '7MinuteWorkout', 'PeriodTracker', 'WaterDrinkReminder',
                    'SleepCycle', 'GoodRx', 'Zombies', 'Fooducate', 'Sweatcoin', 'Habitica',
                   
                  'Shopee', 'Lazada', 'Zalora', 'Zilingo', 'Amazon', 'eBay', 'AliExpress', 'Wish', 'Tokopedia', 'JD.com', 
                   'Walmart', 'Flipkart', 'Alibaba', 'Rakuten', 'Taobao', 'Snapdeal', 'MercadoLibre', 'Daraz', 'Olx', 'Etsy',
                   
                   'googlecanlender','googledrive','samsungcalender','samsungnotes','openai','chatgpt','googledocs',
                'camsoftinformation','googlesheets','wps','notes','camscanner','microsoft','googlesheets','Adobe',
                'Alarm','onedrive','Notisave','calculator','notepad','pdfreader','pdf','filemanger','calender','outlook',
                   
                   'Booking', 'Airbnb', 'Uber', 'Lyft', 'GoogleMaps', 'GoogleEarth', 'TripAdvisor', 'Expedia', 
                   'KAYAK', 'Skyscanner', 'Agoda', 'Hopper', 'Trivago', 'TripIt', 'Traveloka', 'Priceline', 'Orbitz', 'Waze', 'Rome2rio',
                   
                   "NordVPN","SurfShark", "Express","cyberghost","IPVanish","Private Internet Access",
                   "Mullvad","Proton", "Atlas",
                   
                   "Gcash","BPI","Metrobank","iWant","PayMaya","wallet","mocamoca","moca","wise","trust","mrcash",
                   "PayMaya","Coins", "BPI","UnionBank", "CIMB", "Security Bank","BDO","Metrobank","Robinsons",
                    "LANDBANK", "EastWest","PNB", "PSBank", "Maybank2U","Citibank","RCBC", "AXA", "SunLife","AUB",
                   
                   'mobilelegends', 'freefireth', 'spacemafia', 'clashofclans', 'ig', 'shooter', 'bh3global', 'romg', 
                   'gp', 'GenshinImpact', 'fortnite', 'n9', 'clashroyale', 'subwaysurf', 'gardenscapes', 'homescapes', 
                   'candycrushsaga', 'eightballpool', 'mobilehero', 'common', 'legends', 'freefireth', 'spacemafia', 
                   'clashofclans', 'ig', 'shooter', 'romg', 'GenshinImpact', "game","gamecenter",
                   "battle","survival","mcpe","tycoon",
                   
                   'Google News', 'BBC News', 'CNN', 'The New York Times', 'The Guardian', 'Reuters', 'Al Jazeera', 
                   'Fox News', 'NPR News', 'ABC News', 'NBC News', 'Bloomberg', 'CBS News', 'Yahoo News', 'Washington Post', 
                   'HuffPost', 'Associated Press', 'BBC Sport', 'Sky News', 'ESPN', "News",
                   
                   'Spotify', 'Apple Podcasts', 'Google Podcasts', 'Stitcher', 'Podbean', 'TuneIn', 'Pocket Casts', 'Overcast',
                   'Castbox', 'Deezer', 'iHeartRadio', 'RadioPublic', 'Acast', 'Podcast Addict', 'Player FM', 'Breaker', 'SoundCloud', 
                   'Anchor', 'Audible', 'Podyssey',
                   
                   "Netflix", "loklok", "Mivideo", "Youtube", "viu", "Niki", "Disney", "shorttv", "Amazonprime", 
                   "Dramabox", "vivamax", "steam", "Hitv", "HBO", "HDmovies", "smartlivestream"


                  ]}
                
dfdummy = pd.DataFrame(data)
dfdummy['apps'] = dfdummy['apps'].str.lower()
dfdummy = dfdummy.drop_duplicates(keep='last')
len(dfdummy)

234

In [7]:
dfdummy.head()

,category,apps
0,social_media,messanger
1,social_media,facebook
4,social_media,lite
5,social_media,facebooklite
6,social_media,telegram


In [8]:
df.head()

,package_name,part1,part2,part3,part4,part5,last_part,cntuniquecustomer
0,com.tonik.mobile,com,tonik,mobile,None,None,mobile,51447
1,android,android,None,None,None,None,android,51446
2,com.android.providers.blockednumber,com,android,providers,blockednumber,None,blockednumber,51445
3,com.android.location.fused,com,android,location,fused,None,fused,51445
4,com.android.providers.downloads,com,android,providers,downloads,None,downloads,51445


In [9]:
# # Create a list of columns to check for matches
# columns_to_check = ['part1', 'part2', 'part3', 'part4', 'part5', 'last_part']

# # Create a function to check if any part matches the apps
# def match_app(row, apps_set):
#     return any(str(row[col]).lower() in apps_set for col in columns_to_check if pd.notna(row[col]))

# # Convert apps to a set for faster lookup
# apps_set = set(dfdummy['apps'].str.lower())

# # Apply the matching function to create a boolean mask
# mask = df.apply(lambda row: match_app(row, apps_set), axis=1)

# # Use the mask to filter the original dataframe
# df_matched = df[mask]

# # Merge the matched dataframe with dfdummy
# result = pd.merge(df_matched, dfdummy, left_on='last_part', right_on='apps', how='left')

# # If you want to keep all original rows and just add the category where there's a match:
# df['category'] = result['category']
# df['apps'] = result['apps']

# # Display the result

# df.head()

In [10]:
import pandas as pd

# Assuming you already have df and dfdummy dataframes

# Create a list of columns to check for matches
columns_to_check = ['part1', 'part2', 'part3', 'part4', 'part5', 'last_part']

# Create a function to find exact matches
def find_exact_match(row, apps_dict):
    for col in columns_to_check:
        if pd.notna(row[col]) and row[col].lower() in apps_dict:
            return apps_dict[row[col].lower()]
    return pd.NA

# Convert dfdummy to a dictionary for faster lookup
apps_dict = dict(zip(dfdummy['apps'].str.lower(), dfdummy['category']))

# Apply the matching function to create a new 'category' column
df['category'] = df.apply(lambda row: find_exact_match(row, apps_dict), axis=1)

# If you want to add the matched 'apps' column as well
def find_matched_app(row, apps_dict):
    for col in columns_to_check:
        if pd.notna(row[col]) and row[col].lower() in apps_dict:
            return row[col].lower()
    return pd.NA

df['matched_app'] = df.apply(lambda row: find_matched_app(row, apps_dict), axis=1)

# Display the result


In [11]:
df.sample(20)

,package_name,part1,part2,part3,part4,part5,last_part,cntuniquecustomer,category,matched_app
24128,com.tipsyflamingo.flamingocards,com,tipsyflamingo,flamingocards,None,None,flamingocards,4,<NA>,<NA>
76002,com.yachtingpages.app,com,yachtingpages,app,None,None,app,1,<NA>,<NA>
62151,com.insightscs.delivery,com,insightscs,delivery,None,None,delivery,1,<NA>,<NA>
73462,com.headupnav.remotify,com,headupnav,remotify,None,None,remotify,1,<NA>,<NA>
60992,smarter.com.gpscam,smarter,com,gpscam,None,None,gpscam,1,<NA>,<NA>
59114,com.zakasoft.cashreceipt,com,zakasoft,cashreceipt,None,None,cashreceipt,1,<NA>,<NA>
24703,com.hushed.release,com,hushed,release,None,None,release,4,<NA>,<NA>
36096,com.mnhksdev.spidercsim,com,mnhksdev,spidercsim,None,None,spidercsim,2,<NA>,<NA>
39635,com.AlexKosDev.lethal.company.mobile,com,AlexKosDev,lethal,company,mobile,mobile,2,<NA>,<NA>
12186,com.rocket.tools.clean.antivirus.master,com,rocket,tools,clean,antivirus,master,12,<NA>,<NA>


In [12]:
df.to_csv("check.csv")

In [13]:
# Create dfd from the groupby operation
dfd = df.groupby('package_name')['cntuniquecustomer'].sum().reset_index()

# Calculate the total number of unique customers
total_customers = dfd['cntuniquecustomer'].sum()

# Calculate the coverage percentage for each package
dfd['coverage_percentage'] = (dfd['cntuniquecustomer'] / total_customers) * 100

# Categorize packages based on coverage
dfd['coverage_category'] = pd.cut(dfd['coverage_percentage'], 
                                  bins=[0, 5, float('inf')], 
                                  labels=['Below 5%', '5% and Above'],
                                  include_lowest=True)

# Sort the dataframe by coverage percentage in descending order
dfd_sorted = dfd.sort_values('coverage_percentage', ascending=False)

# Display the results
print(dfd_sorted)

# If you want to see the count of packages in each category
category_counts = dfd['coverage_category'].value_counts()
print("\nPackage counts by category:")
print(category_counts)

# If you want to see packages with 5% and above coverage
high_coverage_packages = dfd_sorted[dfd_sorted['coverage_category'] == '5% and Above']
print("\nPackages with 5% and above coverage:")
print(high_coverage_packages)

# Calculate and print the total coverage of high coverage packages
if not high_coverage_packages.empty:
    total_high_coverage = high_coverage_packages['coverage_percentage'].sum()
    print(f"\nTotal coverage of packages with 5% and above coverage: {total_high_coverage:.2f}%")
else:
    print("\nNo packages have 5% or above coverage.")

                                            package_name  cntuniquecustomer  \
57035                                   com.tonik.mobile              51447   
724                                              android              51446   
7355                           com.android.certinstaller              51445   
7505                     com.android.managedprovisioning              51445   
7714                     com.android.providers.telephony              51445   
...                                                  ...                ...   
81971                        ztemt.com.cn.providerpreset                  1   
81972                              zuhapps.recipe.dinner                  1   
81973                         zui.android.overlay.target                  1   
81975                                       zuku.tv.mobi                  1   
81976  zuppitarapps.softwareupdate.appupdatechecker.s...                  1   

       coverage_percentage coverage_category  
5703

In [14]:
dfd.head()

,package_name,cntuniquecustomer,coverage_percentage,coverage_category
0,ACEN.APK,1,0.000007,Below 5%
1,ARRuler.BirdmanStudio.net,1,0.000007,Below 5%
2,Alfasoft.Widgets.PsychedelicAnalogClock,1,0.000007,Below 5%
3,Clean.Sweep.Game,1,0.000007,Below 5%
4,Com.sktelecom.minit,28,0.000192,Below 5%


In [15]:
dfd.to_csv("package.csv", index = False)